In [1]:
#USE_PYGEOS=1
import geopandas as gpd
from pathlib import Path
import pandas as pd
#import pygeos
#gpd.options.use_pygeos = True

/Users/tim/Dropbox/1_Work/ACLU/aclu-nm-legislator-scrape/env/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Input Data Shapefiles

In [5]:
# Senate
input_folder = Path("../gis/NM_Senate")
fp = input_folder / "NM_Senate.shp"
sd = gpd.read_file(fp)

In [6]:
print(sd.crs)
sd.head()

epsg:4269


,DISTRICT,geometry
0,1,"POLYGON ((-108.20971 36.77147, -108.20960 36.7..."
1,10,"POLYGON ((-106.68805 35.13396, -106.68788 35.1..."
2,11,"POLYGON ((-106.76027 35.05588, -106.76061 35.0..."
3,12,"POLYGON ((-106.72010 35.18397, -106.72067 35.1..."
4,13,"POLYGON ((-106.63762 35.04756, -106.63808 35.0..."


In [7]:
# House
input_folder = Path("../gis/NM_House")
fp = input_folder / "NM_House.shp"
hd = gpd.read_file(fp)

In [8]:
hd.head()

,DISTRICT,geometry
0,1,"POLYGON ((-108.22320 36.75468, -108.22343 36.7..."
1,10,"POLYGON ((-106.63944 35.06980, -106.63903 35.0..."
2,11,"POLYGON ((-106.68117 35.08907, -106.68121 35.0..."
3,12,"POLYGON ((-106.72209 35.05312, -106.72204 35.0..."
4,13,"POLYGON ((-106.73786 35.07380, -106.73784 35.0..."


In [10]:
# ZTCA (Zip codes)
input_folder = Path("../gis/NM_ZCTA")
fp = input_folder / "tl_2010_35_zcta510.shp"
zd_raw = gpd.read_file(fp)

In [11]:
zd_raw.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,35,87108,3587108,B5,G6350,S,15224438,0,+35.0724783,-106.5783622,N,"POLYGON ((-106.58628 35.05801, -106.58648 35.0..."
1,35,87104,3587104,B5,G6350,S,11006016,913336,+35.1031834,-106.6755476,N,"POLYGON ((-106.65795 35.09562, -106.65833 35.0..."
2,35,87109,3587109,B5,G6350,S,26199125,183677,+35.1528823,-106.5755359,N,"POLYGON ((-106.57266 35.12975, -106.57764 35.1..."
3,35,87825,3587825,B5,G6350,S,6897450389,2328933,+33.8773166,-107.6562263,N,"POLYGON ((-107.13956 34.30255, -107.13952 34.3..."
4,35,87014,3587014,B5,G6350,S,572886035,772489,+35.2308135,-107.3904488,N,"POLYGON ((-107.20919 35.24598, -107.20916 35.2..."


In [12]:
# get only select columns
zd = zd_raw[['ZCTA5CE10', 'geometry']]
print(zd.crs)
zd.head()

epsg:4269


,ZCTA5CE10,geometry
0,87108,"POLYGON ((-106.58628 35.05801, -106.58648 35.0..."
1,87104,"POLYGON ((-106.65795 35.09562, -106.65833 35.0..."
2,87109,"POLYGON ((-106.57266 35.12975, -106.57764 35.1..."
3,87825,"POLYGON ((-107.13956 34.30255, -107.13952 34.3..."
4,87014,"POLYGON ((-107.20919 35.24598, -107.20916 35.2..."


## Join ZCTA and Districts

In [13]:
# join senate with zcta, senate on left
sx = gpd.sjoin(sd, zd, how = 'left', predicate = 'intersects')[["DISTRICT", "ZCTA5CE10"]]
sx.rename(columns = {'DISTRICT':'senate_district', 
                     'ZCTA5CE10':'ZCTA'}, inplace = True)
sx.head()

,senate_district,ZCTA
0,1,87413
0,1,87499
0,1,87416
0,1,87421
0,1,87417


In [14]:
# join house with zcta
hx = gpd.sjoin(hd, zd, how = 'left', predicate = 'intersects')[["DISTRICT", "ZCTA5CE10"]]
hx.rename(columns = {'DISTRICT':'house_district', 
                     'ZCTA5CE10':'ZCTA'}, inplace = True)
hx.head()

,house_district,ZCTA
0,1,87401
0,1,87402
0,1,87418
0,1,87415
0,1,87410


In [15]:
# join zcta with house and senate
zx1 = gpd.sjoin(zd, hd, how = 'left', predicate = 'intersects')[['ZCTA5CE10','DISTRICT', 'geometry']]
zx1.rename(columns = {'ZCTA5CE10':'ZCTA', 
                      'DISTRICT':'house_district'}, inplace = True)
zx2 = gpd.sjoin(zx1, sd, how = 'left', predicate = 'intersects')[['ZCTA', 'house_district', 'DISTRICT']]
zx2.rename(columns = {'DISTRICT':'senate_district'}, inplace = True)

zx2.head()

,ZCTA,house_district,senate_district
0,87108,10,16
0,87108,10,17
0,87108,10,18
0,87108,19,16
0,87108,19,17


In [16]:
# output 
sx.to_csv('../crosswalks/senate-zcta-crosswalk.csv', index = False)
hx.to_csv('../crosswalks/house-zcta-crosswalk.csv', index = False)

In [27]:
# string concat
(
sx.
    sort_values(['ZCTA']).
    groupby(['senate_district']).agg(', '.join).reset_index().
    to_csv('../crosswalks/senate-zcta-crosswalk-comma-sep.csv', index = False)
)
(
hx.
    sort_values(['ZCTA']).
    groupby(['house_district']).agg(', '.join).reset_index().
    to_csv('../crosswalks/house-zcta-crosswalk-comma-sep.csv', index = False)
)

## Data Exploration

In [ ]:
# count zips per senate district
sx.groupby(['DISTRICT']).size().reset_index(name='COUNT').sort_values(by='COUNT', ascending = False)

In [ ]:
# definitely a many to many relationship here
zx.groupby(['ZCTA5CE10']).size().reset_index(name='COUNT').sort_values(by='COUNT', ascending = False)

In [ ]:
# South Valley, Albuquerque has 8 districts!
zx.query("ZCTA5CE10 == '87105'")